In [3]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pandas as pd
import numpy as np

In [4]:
# Define your custom dataset class
class CustomDataset(Dataset):
    def __init__(self, df_metadata, df_class_dict, base_path):
        self.df_metadata = df_metadata
        self.df_class_dict = df_class_dict
        self.base_path = base_path
        self.transform = transforms.Compose([
            transforms.Resize((256, 256)),  # Resize images to a standard size
            transforms.ToTensor()  # Convert PIL images to tensors
        ])
        
    def __len__(self):
        return len(self.df_metadata)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.base_path, str(self.df_metadata.loc[idx, 'sat_image_path']))  # Convert to string
        mask_path = os.path.join(self.base_path, str(self.df_metadata.loc[idx, 'mask_path']))  # Convert to string
        
        # Load image and mask
        img = Image.open(img_path)
        mask = Image.open(mask_path)
        
        # Apply transformations
        img = self.transform(img)
        mask = np.array(mask)
        mask = torch.from_numpy(mask).long()  # Convert mask to tensor
        
        return img, mask


# Define your segmentation model
class SegmentationModel(nn.Module):
    def __init__(self, num_classes):
        super(SegmentationModel, self).__init__()
        # Define your model architecture here
        # Example:
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.decoder = nn.Sequential(
            nn.Conv2d(64, num_classes, kernel_size=1)  # Output segmentation map with num_classes channels
        )
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


# Load your data
base_path = "../data/segmentation/archive"
df_metadata = pd.read_csv('../data/segmentation/archive/metadata.csv')
df_class_dict = pd.read_csv('../data/segmentation/archive/class_dict.csv')

# Define your model, loss function, and optimizer
model = SegmentationModel(num_classes=len(df_class_dict))
criterion = nn.CrossEntropyLoss()  # Cross-entropy loss is commonly used for segmentation
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Create instances of your dataset and dataloader
dataset = CustomDataset(df_metadata, df_class_dict, base_path)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, masks in dataloader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
    
    epoch_loss = running_loss / len(dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

# Save the trained model
torch.save(model.state_dict(), 'segmentation_model.pth')

 

FileNotFoundError: [Errno 2] No such file or directory: '/Users/julianerath/Documents/Privat/Makeathon/GitHub/Makeathon_Hansea.ai/data/segmentation/archive/nan'